In [23]:
# PARAMS:
IN_FILENAME = './data_clean/abcnews-date-text.csv'
OUT_FILENAME = './data_clean/abcnews-date-text.with-sentiment.csv'
# TL;DR change params & Kernel > Restart & Run All to "run this like a script"

In [24]:
# autoreload
%load_ext autoreload
%autoreload 1

# write code that works in both Python 2 (2.7+) and 3 (3.5+)
from __future__ import absolute_import, division, print_function, unicode_literals
try:
    input = raw_input
    range = xrange
except NameError:
    pass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# !pip install --upgrade google-cloud
# !pip install --upgrade google-cloud-language

In [36]:
import os
import sys
from datetime import datetime
import datetime as dtm
import time
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pprint
pp = pprint.PrettyPrinter(indent=2).pprint
from IPython.display import display as dd
from IPython.display import Image

%matplotlib inline

%aimport lib.visualization
%aimport lib.helpers
%aimport data_cleanup.util
from lib.visualization import simple_plot
import lib.helpers as helpers

np.set_printoptions(threshold=1000)  # 'nan' == all
pd.set_option('display.max_rows', 1000)

In [68]:
data = pd.read_csv(IN_FILENAME, index_col='datetime')
print('> size:', len(data))
dd(data.dtypes)
dd(data.head())
dd(data.tail())

> size: 79


headline_text    object
dtype: object

,headline_text
datetime,
2011-05-14,race call btc cup doomben
2012-05-09,bufferings hopes slump after btc cup draw
2013-04-17,kohler value bitcoin gold
2013-05-11,cummings colt on song in btc cup
2013-10-26,us seizes bitcoins from man linked to silk road


,headline_text
datetime,
2017-12-15,coinbase is going to add altcoins to its platf...
2017-12-15,us prosecutors rush to sell bitcoin seized in ...
2017-12-19,rob pyne turns to bitcoin for post political c...
2017-12-20,north korean hackers raiding bitcoin exchanges
2017-12-23,bitcoin plunges in value after manic upward swing


In [69]:
%env GOOGLE_APPLICATION_CREDENTIALS=/home/neuronq/xxx/ucapstone-dd802778408c.json

env: GOOGLE_APPLICATION_CREDENTIALS=/home/neuronq/xxx/ucapstone-dd802778408c.json


In [70]:
from data_cleanup.util import rate_limit, retry

In [71]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

@retry(tries=4, delay=1)
@rate_limit(n=9, dt=1)
def get_sentiment(text):
    try:
        document = types.Document(
            content=text,
            type=enums.Document.Type.PLAIN_TEXT)
        sentiment = client.analyze_sentiment(document=document).document_sentiment
    except Exception as exc:
        return None, exc
    return sentiment.score, None


In [74]:
data['sentiment'] = 0.0

In [76]:
for idx, headline_text, _ in data.itertuples():
    score, err = get_sentiment(headline_text)
    if not err:
        print("> score for '{}': {}".format(headline_text, score))
        data.loc[idx, ('sentiment',)] = score
    else:
        print("--- ERROR for '{}': {}".format(headline_text, err))

> score for 'race call btc cup doomben': 0.300000011921
> score for 'bufferings hopes slump after btc cup draw': 0.0
> score for 'kohler value bitcoin gold': 0.300000011921
> score for 'cummings colt on song in btc cup': 0.300000011921
> score for 'us seizes bitcoins from man linked to silk road': -0.300000011921
> score for 'bitcoin site hacked founder says': -0.699999988079
> score for 'bitcoin': 0.20000000298
> score for 'british it worker searches landfill for bitcoin fortune': 0.40000000596
> score for 'bitcoins the digital currency explained': 0.40000000596
> score for 'australian dictionary centre chooses bitcoin as word of the year': 0.40000000596
> score for 'zynga adoption pushes bitcoin back over 1000': 0.0
> score for 'vegas casinos accepting bitcoin': 0.600000023842
> score for 'bitcoin exchange operators charged silk road links': 0.300000011921
> score for 'bitcoin rattled': 0.10000000149
> score for 'bitcoins virtual exchange disaster': -0.10000000149
> score for 'an new

In [77]:
data.head()

,headline_text,sentiment
datetime,,
2011-05-14,race call btc cup doomben,0.3
2012-05-09,bufferings hopes slump after btc cup draw,0.0
2013-04-17,kohler value bitcoin gold,0.3
2013-05-11,cummings colt on song in btc cup,0.3
2013-10-26,us seizes bitcoins from man linked to silk road,-0.3


In [78]:
data.to_csv(OUT_FILENAME)